In [5]:
from bs4 import BeautifulSoup
import requests
import json
import re

In [6]:
website_name='Camps Equinunk and Blue Ridge'
website_url='https://www.cecbr.com/'
HEADERS =({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
                   'Accept-Language':'en-US, en;q=0.5'})

home = requests.get(website_url,headers=HEADERS)
home_soup = BeautifulSoup(home.content, 'lxml')

In [7]:
description=home_soup.find('p',attrs={'class':'intro bring-it-in let-it-breathe'}).text.strip()
description

'For generations, Camps Equinunk and Blue Ridge have created a wholesome environment of time-honored traditions and values that, by nurturing simplicity, integrity, kindness and inclusivity, build character. Our unique culture teaches campers and staff to respect themselves, others and the environment and helps them grow into responsible, caring adults and citizens.'

In [8]:
logo_tag = home_soup.find('div', class_='site-branding').find_all('img')
logo_url = logo_tag[1]['src']
print(logo_url)

https://www.cecbr.com/wp-content/themes/cecbr/img/logos.svg


In [18]:
contact_table=home_soup.find('div',attrs={'class':'vcard'})
contact_table
# Extract location
location = contact_table.find('span', class_='street-address').get_text(strip=True)
locality = contact_table.find('span', class_='locality').get_text(strip=True)
region = contact_table.find('abb', class_='region').get_text(strip=True)
location = f"{location}, {locality}, {region}"

zip_code = contact_table.find('span', class_='postal-code').get_text(strip=True)

telephone = contact_table.find('a', class_='tel').get_text(strip=True)

mail='info@cecbr.com'

location= {
        "address": location_full,
        "zip_code": zip_code,
        "latitude": '41.80409',
        "longitude": '-75.18124'
    },
contact= {
        "mail": mail,
        "telephone": telephone
    }
location

({'address': 'P.O. Box 365, Equinunk, PA',
  'zip_code': '18417',
  'latitude': '41.80409',
  'longitude': '-75.18124'},)

In [20]:
# Initialize an empty list to store social media links
socials = []

# Locate the social media section
social_bar = contact_table.select_one('div.social')
if social_bar:
    # Find all <a> tags within the social icons list
    social_links = social_bar.find_all('a')
    for link in social_links:
        href = link.get('href', '').strip()  # Get the hyperlink and strip whitespace

        # Check the URL for known platform names and append to the socials list
        if 'facebook' in href:
            socials.append({"facebook": href})
        elif 'twitter' in href:
            socials.append({"twitter": href})
        elif 'youtube' in href:
            socials.append({"youtube": href})
        elif 'instagram' in href:
            socials.append({"instagram": href})
        elif 'blog' in href:
            socials.append({"blog": href})

website_url = "https://www.cecbr.com"
for item in socials:
    for key, value in item.items():
        if value.startswith('/'):
            item[key] = website_url + value

socials


[{'facebook': 'https://www.facebook.com/CECBR'},
 {'instagram': 'https://instagram.com/equinunkblueridge'},
 {'youtube': 'https://www.youtube.com/channel/UCIQb-JOJ8huzMAjedwvPE2Q'},
 {'blog': 'https://www.cecbr.com/blog/'}]

In [21]:
images_url = []

img_tags = home_soup.find_all('img')

for img in img_tags:
    src = img.get('src', '')
    if src and src.startswith("http") and (".jpeg" in src or ".jpg" in src or ".webp" in src):
        images_url.append(src)
images_url

['https://www.cecbr.com/wp-content/themes/cecbr/img/tiles/tile-lg-b-brother-sister-experience.jpg',
 'https://www.cecbr.com/wp-content/themes/cecbr/img/tiles/tile-lg-b-blue-ridge.jpg',
 'https://www.cecbr.com/wp-content/themes/cecbr/img/tiles/tile-lg-b-equinunk.jpg',
 'https://www.cecbr.com/wp-content/themes/cecbr/img/tiles/tile-lg-b-dates-fees-faqs.jpg',
 'https://www.cecbr.com/wp-content/themes/cecbr/img/tiles/tile-lg-b-special-events.jpg',
 'https://www.cecbr.com/wp-content/themes/cecbr/img/tiles/tile-lg-b-tour-camp.jpg',
 'https://www.cecbr.com/wp-content/themes/cecbr/img/gaga-home-tile.jpg?v=2024',
 'https://www.cecbr.com/wp-content/themes/cecbr/img/tiles/ebr-tile-winter-reunion-2024.jpg?v=2024',
 'https://www.cecbr.com/wp-content/themes/cecbr/img/tiles/tile-lg-b-work-at-camp.jpg']

In [22]:
rd_url = 'https://www.cecbr.com/about-ebr-camps/dates-fees-faqs/'
rd = requests.get(rd_url, headers=HEADERS)
rd_soup = BeautifulSoup(rd.content, 'lxml')

In [23]:
rd_table= rd_soup.find_all('div', attrs={'class': 'wp-block-columns is-layout-flex wp-container-core-columns-is-layout-1 wp-block-columns-is-layout-flex'})
rd_table

[<div class="wp-block-columns is-layout-flex wp-container-core-columns-is-layout-1 wp-block-columns-is-layout-flex"><div class="wp-block-column dates is-layout-flow wp-block-column-is-layout-flow" id="cecbr-dates" style="flex-basis:60%"><h2 class="wp-block-heading">2025 Season</h2><p class="has-text-align-center intro"><strong>New Family Picnic &amp; Meeting</strong><br/><em>Saturday, June 7th 2025</em></p><p class="has-text-align-center intro"><strong>First Day of Camp</strong><br/><em>Sunday, June 29th 2025</em></p><p class="has-text-align-center intro"><strong>Visiting Day</strong><br/><em>Saturday, July 19th 2025</em></p><p class="has-text-align-center intro"><strong>Last Day of Camp</strong><br/><em>Saturday, August 16th 2025</em></p></div><div class="wp-block-column tuition is-layout-flow wp-block-column-is-layout-flow" style="flex-basis:40%"><h2 class="wp-block-heading">2025 Fees</h2><p class="has-text-align-center rate"><strong>$16,400</strong><br/>Tuition for enrollments recei

In [24]:
# Locate the relevant section for 2025 Season
season_section = rd_table[0]  # Assuming it's the first div in rd_table

# Extract start and end dates
dates_section = season_section.find('div', class_='dates')
start_date = None
end_date = None
if dates_section:
    events = dates_section.find_all('p', class_='intro')
    for event in events:
        name = event.find('strong').get_text(strip=True)
        date = event.find('em').get_text(strip=True) if event.find('em') else None
        if "First Day" in name:
            start_date = date
        elif "Last Day" in name:
            end_date = date

# Extract tuition cost
fees_section = season_section.find('div', class_='tuition')
cost = None
if fees_section:
    rates = fees_section.find_all('p', class_='rate')
    for rate in rates:
        if "Tuition for enrollments received by" in rate.get_text(strip=True):
            match = re.search(r'\$\d{1,3}(?:,\d{3})?', rate.get_text(strip=True))
            if match:
                cost = match.group(0)
                break  # Stop after getting the first relevant tuition rate

# Construct the desired dictionary
season_details = {
    "name": "2025 Season",
    "duration": None,
    "cost": cost,
    "start_date": start_date,
    "end_date": end_date,
    "location": "Equinunk, PA",
    "deposits": None,
    "eligible": None
}

# Output the details
import json
print(json.dumps(season_details, indent=4))


{
    "name": "2025 Season",
    "duration": null,
    "cost": "$16,400",
    "start_date": "Sunday, June 29th 2025",
    "end_date": "Saturday, August 16th 2025",
    "location": "Equinunk, PA",
    "deposits": null,
    "eligible": null
}


In [25]:
cat_url = 'https://www.cecbr.com/about-ebr-camps/activities-facilities/'
cat = requests.get(cat_url, headers=HEADERS)
cat_soup = BeautifulSoup(cat.content, 'lxml')

In [26]:
categories = []

category_table = cat_soup.find('div', attrs={'id': 'modal-ready'})

# Iterate through each category block
category_blocks = category_table.find_all(['h2', 'p'])

for block in category_blocks:
    if block.name == 'h2' and 'wp-block-heading' in block.get('class', []):
        category_name = block.get_text(strip=True)  # Extract the category name

        # Look for the next <p> tag which should be the description
        category_desc = None
        next_sibling = block.find_next_sibling('p')
        if next_sibling:
            category_desc = next_sibling.get_text(strip=True)  # Extract description


        categories.append({
            'name': category_name,
            'description': category_desc,
            'image': 'null'
        })

categories

[{'name': 'Athletics',
  'description': 'Our athletic programs and facilities at Equinunk and Blue Ridge are top notch. Our coaches and instructors are highly credentialed professionals, and work throughout the year with high school and college athletes. Their talent is magnified in the camp setting by their ability to relate to your children and make learning simple and fun. Our CECBR campus features several sport facilities lit for night play so campers can enjoy our numerous athletic fields, courts, and gymnasiums in the evening!',
  'image': 'null'},
 {'name': 'Waterfront',
  'description': 'Our waterfront facilities are the “cool” places to be on warm summer days and are supervised by our experienced Red Cross certified lifeguards. Both our Equinunk and Blue Ridge campuses feature heated pools with saltwater filtration systems, and underwater sound systems and lights for night swims! The Waterfront, set on Union Lake, is home to our waterski program, which is incredibly popular am

In [27]:
activities=[]
activities_table = cat_soup.find_all('div', attrs={'class': 'activities-list'})
activities_table
for i in activities_table:
    list=i.find_all('li')
    for j in list:
        activity=j.text.strip()
        activities.append({'name':activity,'description':'null','image':'null'})

activities

[{'name': 'Baseball', 'description': 'null', 'image': 'null'},
 {'name': 'Basketball', 'description': 'null', 'image': 'null'},
 {'name': 'Field Hockey', 'description': 'null', 'image': 'null'},
 {'name': 'Fitness', 'description': 'null', 'image': 'null'},
 {'name': 'Flag Football', 'description': 'null', 'image': 'null'},
 {'name': 'Frisbee Golf', 'description': 'null', 'image': 'null'},
 {'name': 'GAGA', 'description': 'null', 'image': 'null'},
 {'name': 'Golf', 'description': 'null', 'image': 'null'},
 {'name': 'Gymnastics', 'description': 'null', 'image': 'null'},
 {'name': 'Lacrosse', 'description': 'null', 'image': 'null'},
 {'name': 'Roller Hockey', 'description': 'null', 'image': 'null'},
 {'name': 'Soccer', 'description': 'null', 'image': 'null'},
 {'name': 'Softball', 'description': 'null', 'image': 'null'},
 {'name': 'Street Hockey', 'description': 'null', 'image': 'null'},
 {'name': 'Team Handball', 'description': 'null', 'image': 'null'},
 {'name': 'Tennis', 'description':

In [28]:
Cebr= [{
    'name': website_name,  
    'url': website_url,
    'logo':logo_url,
    'description':description,
    'location': location,
    'contact': contact,
    'socials': socials,
    'image_urls':images_url,
    'sessions': season_details,
    'categories': categories,
    'activities': activities
}]
file_path = 'cebr.json'

# Export the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(Cebr, json_file, indent=4)